<a href="https://colab.research.google.com/github/YiHua00000/Fraud-detection-project/blob/main/Copy_of_summer_project_frauddetect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd //content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
!ls "/content/drive/MyDrive"

'adjusted query.gdoc'
'Colab Notebooks'
'Copy of  Project_0503_02_Presentation.pptx.gslides'
'--CREATE Statements.gdoc'
'image project'
'Oigetit - Job Application.gdoc'
'ppp_applicants_ga_full (1).csv'
 ppp_applicants_ga_full.csv
 ppp-removed-ga.xlsx
 query.gdoc
 View_adjustment.gdoc
'Yi_Hua_Huang_resume (1).pdf'
 Yi_Hua_Huang_resume.pdf
 Yi_Hua_Huang_Resume.pdf


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as datetime
from dateutil.parser import parse
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import svm
from xgboost import XGBClassifier

# About the dataset
Abstract: PPloans from GA in 2020.

Data Set Information: PPP loans are issued by private lenders and credit unions, and then they are backed by the Small Business Administration (SBA). The basic purpose of the PPP is to incentivize small businesses to keep workers on payroll and/or to rehire laid-off workers that lost wages due to COVID-19 disruptions.

In [ ]:
full_data = pd.read_csv('ppp_applicants_ga_full.csv')
removed_data = pd.read_excel('ppp-removed-ga.xlsx')


# Processing data

In [ ]:
removed_data

,name,amount,state,address,city,zip,naics_code,business_type,jobs_retained,date_approved,...,project_county_name,project_state,project_zip,originating_lender_city,originating_lender_state,loan_status_date,originating_lender_location_id,lmi_indicator,forgiveness_amount,forgiveness_date
0,TRIPLE C CUSTOM CONCRETE LLC,43655.0,GA,106 Lisa Street East Dublin GA,East Dublin,31027.0,238990.0,Limited Liability Company(LLC),7.0,2021-05-11,...,LAURENS,GA,31027.0,Saint Petersburg,FL,2021-05-11,123499.0,Y,NaN,NaN
1,DAVIS LAWN CARE SERVICES,35000.0,GA,752 Savannah Ave N/A,Waynesboro,30830-1156,112910.0,Partnership,2.0,2021-05-12,...,BURKE,GA,30830-1156,ARLINGTON,MA,2021-05-12,262380.0,N,NaN,NaN
2,LONNIE RATLIFF,28949.0,GA,55 Maple St NW Apt 802,Atlanta,30314-4760,531390.0,Single Member LLC,5.0,2021-04-16,...,FULTON,GA,30314-4760,PHOENIXVILLE,PA,2021-04-16,121536.0,Y,NaN,NaN
3,DENARD TRIMBLE,20833.0,GA,727 Chapman St,Jonesboro,30238-5681,812111.0,Sole Proprietorship,1.0,2021-05-22,...,FAYETTE,GA,30238-5681,Brea,CA,2021-05-22,514546.0,Y,NaN,NaN
4,DANA VINES,20833.0,GA,3548 Tracey Dr,Duluth,30096-3861,311999.0,Sole Proprietorship,1.0,2021-03-20,...,GWINNETT,GA,30096-3861,Miami,FL,2021-03-20,529192.0,N,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25831,DONNELL WILSON,1950.0,GA,1000 Paces Ln,Woodstock,30189-4829,621511.0,Sole Proprietorship,1.0,2021-04-01,...,CHEROKEE,GA,30189-4829,Laguna Hills,CA,2021-04-01,509316.0,N,NaN,NaN
25832,MAMADOU DIOULDE DIALLO,1849.0,GA,7000 Roswell Rd Apt 252,Atlanta,30328-2389,485310.0,Independent Contractors,1.0,2021-05-21,...,FULTON,GA,30328-2389,Bedford,TX,2021-05-21,529472.0,N,NaN,NaN
25833,ANTHONY HARRIS,1245.0,GA,2849 Blount St,Atlanta,30344-3732,485310.0,Independent Contractors,1.0,2021-04-08,...,FULTON,GA,30344-3732,Bedford,TX,2021-04-08,529472.0,Y,NaN,NaN
25834,TAAHIRA ROBINSON,1075.0,GA,8541 Glendevon Ct,Riverdale,30274-4573,561110.0,Limited Liability Company(LLC),1.0,2021-05-11,...,CLAYTON,GA,30274-4573,Miami,FL,2021-05-11,529192.0,Y,NaN,NaN


In [ ]:
full_data

,name,amount,state,address,city,zip,naics_code,business_type,jobs_retained,date_approved,...,project_county_name,project_state,project_zip,originating_lender_city,originating_lender_state,loan_status_date,originating_lender_location_id,lmi_indicator,forgiveness_amount,forgiveness_date
0,LUMMUS CORPORATION,2000000.00,GA,225 Bourne Blvd,Savannah,31408-9586,333249.0,Corporation,294,2021-02-20,...,CHATHAM,GA,31408-9586,CHICAGO,IL,NaN,116998,N,NaN,NaN
1,COLIANT SOLUTIONS INC.,1294555.00,GA,2703 Brickton North Dr,Buford,30518-9101,541519.0,Corporation,63,2021-01-31,...,GWINNETT,GA,30518-9101,COLUMBUS,GA,NaN,19248,Y,NaN,NaN
2,YOHE PLUMBING INC,729509.00,GA,1120 Franke Court N/A,Augusta,30909,238220.0,Corporation,105,2021-03-12,...,RICHMOND,GA,30909,PHOENIXVILLE,PA,NaN,121536,N,NaN,NaN
3,LEWIS COLOR LITHOGRAPHERS INC,571193.40,GA,30 Joe Kennedy Blvd,Statesboro,30458-3417,323111.0,Corporation,49,2021-02-02,...,BULLOCH,GA,30458-3417,COLUMBUS,GA,NaN,19248,N,NaN,NaN
4,"ALMA PAK INTERNATIONAL, LLC",472700.00,GA,230 PINEVIEW RD,ALMA,31510-4326,445230.0,Partnership,37,2020-04-10,...,BACON,GA,31510-4326,ALMA,GA,2021-01-08,17811,N,475877.59,2020-12-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553823,PAUL CLARK,757.00,GA,609 Fern St,Waycross,31501-3210,561492.0,Independent Contractors,1,2021-04-30,...,WARE,GA,31501-3210,Miami,FL,NaN,529192,N,NaN,NaN
553824,DEWEY GRIFFIN,700.00,GA,245 Moray Rd,Alma,31510-4371,111940.0,Sole Proprietorship,1,2021-04-02,...,BACON,GA,31510-4371,ALMA,GA,2021-09-24,17808,N,702.61,2021-08-23
553825,HAYDEN WHITLOW,582.71,GA,2696 Douglas Ct NW,Acworth,30101-3677,561730.0,Independent Contractors,1,2021-02-19,...,COBB,GA,30101-3677,HINTON,OK,NaN,58771,N,NaN,NaN
553826,KEVIN LEWIS,270.00,GA,4830 Northway Drive Northeast,Sandy Springs,30342-2424,541990.0,Sole Proprietorship,1,2020-08-03,...,FULTON,GA,30342-2424,TEANECK,NJ,NaN,456756,N,NaN,NaN


In [ ]:
removed_data.columns

Index(['name', 'amount', 'state', 'address', 'city', 'zip', 'naics_code',
       'business_type', 'jobs_retained', 'date_approved', 'lender',
       'congressional_district', 'loan_number', 'sba_office_code',
       'processing_method', 'loan_status', 'term', 'sba_guaranty_percentage',
       'initial_approval_amount', 'current_approval_amount',
       'undisbursed_amount', 'servicing_lender_location_id',
       'servicing_lender_name', 'servicing_lender_address',
       'servicing_lender_city', 'servicing_lender_state',
       'servicing_lender_zip', 'rural_urban_indicator', 'hubzone_indicator',
       'business_age_description', 'project_city', 'project_county_name',
       'project_state', 'project_zip', 'originating_lender_city',
       'originating_lender_state', 'loan_status_date',
       'originating_lender_location_id', 'lmi_indicator', 'forgiveness_amount',
       'forgiveness_date'],
      dtype='object')

In [ ]:
full_data.columns

Index(['name', 'amount', 'state', 'address', 'city', 'zip', 'naics_code',
       'business_type', 'jobs_retained', 'date_approved', 'lender',
       'congressional_district', 'loan_number', 'sba_office_code',
       'processing_method', 'loan_status', 'term', 'sba_guaranty_percentage',
       'initial_approval_amount', 'current_approval_amount',
       'undisbursed_amount', 'servicing_lender_location_id',
       'servicing_lender_name', 'servicing_lender_address',
       'servicing_lender_city', 'servicing_lender_state',
       'servicing_lender_zip', 'rural_urban_indicator', 'hubzone_indicator',
       'business_age_description', 'project_city', 'project_county_name',
       'project_state', 'project_zip', 'originating_lender_city',
       'originating_lender_state', 'loan_status_date',
       'originating_lender_location_id', 'lmi_indicator', 'forgiveness_amount',
       'forgiveness_date'],
      dtype='object')

In [ ]:
removed_data['Removed']=np.where(removed_data['name']!=0, True, False)

In [ ]:
removed_data['loan_number'].value_counts()

8.165379e+09    1
9.777059e+09    1
9.134169e+09    1
7.007449e+09    1
5.285748e+09    1
               ..
8.395809e+09    1
1.495939e+09    1
8.572469e+09    1
7.022719e+09    1
6.722089e+09    1
Name: loan_number, Length: 25836, dtype: int64

In [ ]:
removed_data.columns

Index(['name', 'amount', 'state', 'address', 'city', 'zip', 'naics_code',
       'business_type', 'jobs_retained', 'date_approved', 'lender',
       'congressional_district', 'loan_number', 'sba_office_code',
       'processing_method', 'loan_status', 'term', 'sba_guaranty_percentage',
       'initial_approval_amount', 'current_approval_amount',
       'undisbursed_amount', 'servicing_lender_location_id',
       'servicing_lender_name', 'servicing_lender_address',
       'servicing_lender_city', 'servicing_lender_state',
       'servicing_lender_zip', 'rural_urban_indicator', 'hubzone_indicator',
       'business_age_description', 'project_city', 'project_county_name',
       'project_state', 'project_zip', 'originating_lender_city',
       'originating_lender_state', 'loan_status_date',
       'originating_lender_location_id', 'lmi_indicator', 'forgiveness_amount',
       'forgiveness_date', 'Removed'],
      dtype='object')

In [ ]:
combined_data=full_data.append(removed_data)

In [ ]:
combined_data

,name,amount,state,address,city,zip,naics_code,business_type,jobs_retained,date_approved,...,project_state,project_zip,originating_lender_city,originating_lender_state,loan_status_date,originating_lender_location_id,lmi_indicator,forgiveness_amount,forgiveness_date,Removed
0,LUMMUS CORPORATION,2000000.0,GA,225 Bourne Blvd,Savannah,31408-9586,333249.0,Corporation,294.0,2021-02-20,...,GA,31408-9586,CHICAGO,IL,NaN,116998.0,N,NaN,NaN,NaN
1,COLIANT SOLUTIONS INC.,1294555.0,GA,2703 Brickton North Dr,Buford,30518-9101,541519.0,Corporation,63.0,2021-01-31,...,GA,30518-9101,COLUMBUS,GA,NaN,19248.0,Y,NaN,NaN,NaN
2,YOHE PLUMBING INC,729509.0,GA,1120 Franke Court N/A,Augusta,30909,238220.0,Corporation,105.0,2021-03-12,...,GA,30909,PHOENIXVILLE,PA,NaN,121536.0,N,NaN,NaN,NaN
3,LEWIS COLOR LITHOGRAPHERS INC,571193.4,GA,30 Joe Kennedy Blvd,Statesboro,30458-3417,323111.0,Corporation,49.0,2021-02-02,...,GA,30458-3417,COLUMBUS,GA,NaN,19248.0,N,NaN,NaN,NaN
4,"ALMA PAK INTERNATIONAL, LLC",472700.0,GA,230 PINEVIEW RD,ALMA,31510-4326,445230.0,Partnership,37.0,2020-04-10,...,GA,31510-4326,ALMA,GA,2021-01-08,17811.0,N,475877.59,2020-12-15,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25831,DONNELL WILSON,1950.0,GA,1000 Paces Ln,Woodstock,30189-4829,621511.0,Sole Proprietorship,1.0,2021-04-01 00:00:00,...,GA,30189-4829,Laguna Hills,CA,2021-04-01 00:00:00,509316.0,N,NaN,NaN,True
25832,MAMADOU DIOULDE DIALLO,1849.0,GA,7000 Roswell Rd Apt 252,Atlanta,30328-2389,485310.0,Independent Contractors,1.0,2021-05-21 00:00:00,...,GA,30328-2389,Bedford,TX,2021-05-21 00:00:00,529472.0,N,NaN,NaN,True
25833,ANTHONY HARRIS,1245.0,GA,2849 Blount St,Atlanta,30344-3732,485310.0,Independent Contractors,1.0,2021-04-08 00:00:00,...,GA,30344-3732,Bedford,TX,2021-04-08 00:00:00,529472.0,Y,NaN,NaN,True
25834,TAAHIRA ROBINSON,1075.0,GA,8541 Glendevon Ct,Riverdale,30274-4573,561110.0,Limited Liability Company(LLC),1.0,2021-05-11 00:00:00,...,GA,30274-4573,Miami,FL,2021-05-11 00:00:00,529192.0,Y,NaN,NaN,True


In [ ]:
combined_data['Removed'].value_counts()

True    25836
Name: Removed, dtype: int64

In [ ]:
combined_data['Removed'].fillna(False,inplace=True)

In [ ]:
combined_data['Removed'].value_counts()

False    553828
True      25836
Name: Removed, dtype: int64

In [ ]:
combined_data=combined_data.reset_index()

In [ ]:
combined_data.columns[0]

'index'

In [ ]:
combined_data=combined_data.drop(combined_data.columns[0], axis=1)
combined_data

,name,amount,state,address,city,zip,naics_code,business_type,jobs_retained,date_approved,...,project_state,project_zip,originating_lender_city,originating_lender_state,loan_status_date,originating_lender_location_id,lmi_indicator,forgiveness_amount,forgiveness_date,Removed
0,LUMMUS CORPORATION,2000000.0,GA,225 Bourne Blvd,Savannah,31408-9586,333249.0,Corporation,294.0,2021-02-20,...,GA,31408-9586,CHICAGO,IL,NaN,116998.0,N,NaN,NaN,False
1,COLIANT SOLUTIONS INC.,1294555.0,GA,2703 Brickton North Dr,Buford,30518-9101,541519.0,Corporation,63.0,2021-01-31,...,GA,30518-9101,COLUMBUS,GA,NaN,19248.0,Y,NaN,NaN,False
2,YOHE PLUMBING INC,729509.0,GA,1120 Franke Court N/A,Augusta,30909,238220.0,Corporation,105.0,2021-03-12,...,GA,30909,PHOENIXVILLE,PA,NaN,121536.0,N,NaN,NaN,False
3,LEWIS COLOR LITHOGRAPHERS INC,571193.4,GA,30 Joe Kennedy Blvd,Statesboro,30458-3417,323111.0,Corporation,49.0,2021-02-02,...,GA,30458-3417,COLUMBUS,GA,NaN,19248.0,N,NaN,NaN,False
4,"ALMA PAK INTERNATIONAL, LLC",472700.0,GA,230 PINEVIEW RD,ALMA,31510-4326,445230.0,Partnership,37.0,2020-04-10,...,GA,31510-4326,ALMA,GA,2021-01-08,17811.0,N,475877.59,2020-12-15,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579659,DONNELL WILSON,1950.0,GA,1000 Paces Ln,Woodstock,30189-4829,621511.0,Sole Proprietorship,1.0,2021-04-01 00:00:00,...,GA,30189-4829,Laguna Hills,CA,2021-04-01 00:00:00,509316.0,N,NaN,NaN,True
579660,MAMADOU DIOULDE DIALLO,1849.0,GA,7000 Roswell Rd Apt 252,Atlanta,30328-2389,485310.0,Independent Contractors,1.0,2021-05-21 00:00:00,...,GA,30328-2389,Bedford,TX,2021-05-21 00:00:00,529472.0,N,NaN,NaN,True
579661,ANTHONY HARRIS,1245.0,GA,2849 Blount St,Atlanta,30344-3732,485310.0,Independent Contractors,1.0,2021-04-08 00:00:00,...,GA,30344-3732,Bedford,TX,2021-04-08 00:00:00,529472.0,Y,NaN,NaN,True
579662,TAAHIRA ROBINSON,1075.0,GA,8541 Glendevon Ct,Riverdale,30274-4573,561110.0,Limited Liability Company(LLC),1.0,2021-05-11 00:00:00,...,GA,30274-4573,Miami,FL,2021-05-11 00:00:00,529192.0,Y,NaN,NaN,True


In [ ]:
combined_data.columns

Index(['name', 'amount', 'state', 'address', 'city', 'zip', 'naics_code',
       'business_type', 'jobs_retained', 'date_approved', 'lender',
       'congressional_district', 'loan_number', 'sba_office_code',
       'processing_method', 'loan_status', 'term', 'sba_guaranty_percentage',
       'initial_approval_amount', 'current_approval_amount',
       'undisbursed_amount', 'servicing_lender_location_id',
       'servicing_lender_name', 'servicing_lender_address',
       'servicing_lender_city', 'servicing_lender_state',
       'servicing_lender_zip', 'rural_urban_indicator', 'hubzone_indicator',
       'business_age_description', 'project_city', 'project_county_name',
       'project_state', 'project_zip', 'originating_lender_city',
       'originating_lender_state', 'loan_status_date',
       'originating_lender_location_id', 'lmi_indicator', 'forgiveness_amount',
       'forgiveness_date', 'Removed'],
      dtype='object')

# **Numeric variables**

In [ ]:
combined_data['amount'].isna().sum()

0

In [ ]:
combined_data['jobs_retained'].isna().sum()

0

In [ ]:
combined_data['amount_per_join']=np.where(combined_data['jobs_retained']>0, (combined_data['amount']/ combined_data['jobs_retained']).round(2), combined_data['amount'])

In [ ]:
combined_data['amount_per_join']


0          6802.72
1         20548.49
2          6947.70
3         11657.01
4         12775.68
            ...   
579659     1950.00
579660     1849.00
579661     1245.00
579662     1075.00
579663      787.00
Name: amount_per_join, Length: 579664, dtype: float64

In [ ]:
combined_data['amount_per_join'].describe()



count    5.796640e+05
mean     1.303513e+04
std      8.071921e+03
min      1.000000e+00
25%      5.994840e+03
50%      1.321300e+04
75%      2.074300e+04
max      1.253112e+06
Name: amount_per_join, dtype: float64

In [ ]:
combined_data['date_approved']

0                  2021-02-20
1                  2021-01-31
2                  2021-03-12
3                  2021-02-02
4                  2020-04-10
                 ...         
579659    2021-04-01 00:00:00
579660    2021-05-21 00:00:00
579661    2021-04-08 00:00:00
579662    2021-05-11 00:00:00
579663    2021-05-23 00:00:00
Name: date_approved, Length: 579664, dtype: object

#### **Date**

In [ ]:
combined_data['Date_launched'] = pd.to_datetime(combined_data['date_approved']).dt.date

In [ ]:
combined_data['Date_launched'].describe()

count         579664
unique           237
top       2020-05-01
freq           22403
Name: Date_launched, dtype: object

In [ ]:
combined_data['launched_year'] = pd.DatetimeIndex(combined_data['Date_launched']).year
combined_data['launched_month'] = pd.DatetimeIndex(combined_data['Date_launched']).month

In [ ]:
combined_data['term'].isna().sum()

0

In [ ]:
combined_data['undisbursed_amount'].describe()

count    5.795480e+05
mean     8.255126e+02
std      1.494828e+04
min     -1.035920e+05
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.000000e+07
Name: undisbursed_amount, dtype: float64

In [ ]:
meanVal = combined_data['undisbursed_amount'].mean()

In [ ]:
combined_data['undisbursed_amount'].fillna(value=meanVal, inplace=True)

In [ ]:
combined_data['Difference_current_initial_amount']=combined_data['current_approval_amount']-combined_data['initial_approval_amount']


In [ ]:
numeric_data = pd.DataFrame(combined_data,columns=['amount','jobs_retained','amount_per_join', 'term', 'undisbursed_amount', 'Difference_current_initial_amount'])
corrMatrix = numeric_data.corr().abs()

print(corrMatrix)

                                     amount  jobs_retained  amount_per_join  \
amount                             1.000000       0.819283         0.023514   
jobs_retained                      0.819283       1.000000         0.125550   
amount_per_join                    0.023514       0.125550         1.000000   
term                               0.170683       0.224632         0.294818   
undisbursed_amount                 0.068899       0.028251         0.046451   
Difference_current_initial_amount  0.008997       0.023807         0.007638   

                                       term  undisbursed_amount  \
amount                             0.170683            0.068899   
jobs_retained                      0.224632            0.028251   
amount_per_join                    0.294818            0.046451   
term                               1.000000            0.029909   
undisbursed_amount                 0.029909            1.000000   
Difference_current_initial_amount  0.014777 

In [ ]:

upper = corrMatrix.where(np.triu(np.ones(corrMatrix.shape), k=1).astype(np.bool_))

to_drop = [column for column in upper.columns if any(upper[column] > 0.5)]


numeric_data.drop(to_drop, axis=1, inplace=True)


In [ ]:
numeric_data

,amount,amount_per_join,term,undisbursed_amount,Difference_current_initial_amount
0,2000000.0,6802.72,60.0,0.0,0.0
1,1294555.0,20548.49,60.0,0.0,0.0
2,729509.0,6947.70,60.0,0.0,0.0
3,571193.4,11657.01,60.0,0.0,-0.6
4,472700.0,12775.68,24.0,0.0,0.0
...,...,...,...,...,...
579659,1950.0,1950.00,60.0,1950.0,0.0
579660,1849.0,1849.00,60.0,1849.0,0.0
579661,1245.0,1245.00,60.0,1245.0,0.0
579662,1075.0,1075.00,60.0,1075.0,0.0




```
# This is formatted as code
```

# **Categorical variables**

In [ ]:
combined_data['launched_month'].mode()

0    4
dtype: int64

In [ ]:
##categorical_data = pd.DataFrame(combined_data,columns=['loan_number','launched_month','launched_year','naics_code_name', 'business_type', 'city_name_top', 'servicing_lender_top', 'servicing_lender_state_top', 'servicing_lender_city_top', 'project_city_top', 'project_county_top'
##, 'lender_state_top', 'lender_city_top', 'lender_servicing_lender_state_city'])00000

In [ ]:
combined_data['naics_code'].isna().sum()

2340

In [ ]:
combined_data['naics_code']=combined_data['naics_code'].fillna(10)

In [ ]:
naics_code_name=[]
for i in combined_data['naics_code']:

  if(int(str(i)[:2]) == 11):
        naics_code_name.append('Agriculture, Forestry, Fishing and Hunting')

  elif(int(str(i)[:2]) == 21):
        naics_code_name.append('Mining')
  elif(int(str(i)[:2]) == 22):
        naics_code_name.append('Utilities')
  elif(int(str(i)[:2]) == 23):
        naics_code_name.append('Construction')
  elif(int(str(i)[:2]) == 31):
        naics_code_name.append('Manufacturing')
  elif(int(str(i)[:2]) == 32):
        naics_code_name.append('Manufacturing')
  elif(int(str(i)[:2]) == 33):
        naics_code_name.append('Manufacturing')
  elif(int(str(i)[:2]) == 42):
        naics_code_name.append('Wholesale Trade')
  elif(int(str(i)[:2]) == 44):
        naics_code_name.append('Retail Trade')
  elif(int(str(i)[:2]) == 45):
        naics_code_name.append('Retail Trade')
  elif(int(str(i)[:2]) == 48):
        naics_code_name.append('Transportation and Warehousing')
  elif(int(str(i)[:2]) == 49):
        naics_code_name.append('Transportation and Warehousing')
  elif(int(str(i)[:2]) == 51):
        naics_code_name.append('Information')
  elif(int(str(i)[:2]) == 52):
        naics_code_name.append('Finance and Insurance')
  elif(int(str(i)[:2]) == 53):
        naics_code_name.append('Real Estate Rental and Leasing')
  elif(int(str(i)[:2]) == 54):
        naics_code_name.append('Professional, Scientific, and Technical Services')
  elif(int(str(i)[:2]) == 55):
        naics_code_name.append('Management of Companies and Enterprises')
  elif(int(str(i)[:2]) == 56):
        naics_code_name.append('Administrative and Support and Waste Management and Remediation Services')
  elif(int(str(i)[:2]) == 61):
        naics_code_name.append('Educational Services')
  elif(int(str(i)[:2]) == 62):
        naics_code_name.append('Health Care and Social Assistance')
  elif(int(str(i)[:2]) == 71):
        naics_code_name.append('Arts, Entertainment, and Recreation')
  elif(int(str(i)[:2]) == 72):
        naics_code_name.append('Accommodation and Food Services')
  elif(int(str(i)[:2]) == 81):
        naics_code_name.append('Other Services')
  elif(int(str(i)[:2]) == 92):
        naics_code_name.append('Public Administration')
  else:
    naics_code_name.append('Others')

In [ ]:
combined_data['naics_code_name']=naics_code_name

In [ ]:
combined_data.columns

Index(['name', 'amount', 'state', 'address', 'city', 'zip', 'naics_code',
       'business_type', 'jobs_retained', 'date_approved', 'lender',
       'congressional_district', 'loan_number', 'sba_office_code',
       'processing_method', 'loan_status', 'term', 'sba_guaranty_percentage',
       'initial_approval_amount', 'current_approval_amount',
       'undisbursed_amount', 'servicing_lender_location_id',
       'servicing_lender_name', 'servicing_lender_address',
       'servicing_lender_city', 'servicing_lender_state',
       'servicing_lender_zip', 'rural_urban_indicator', 'hubzone_indicator',
       'business_age_description', 'project_city', 'project_county_name',
       'project_state', 'project_zip', 'originating_lender_city',
       'originating_lender_state', 'loan_status_date',
       'originating_lender_location_id', 'lmi_indicator', 'forgiveness_amount',
       'forgiveness_date', 'Removed', 'amount_per_join', 'Date_launched',
       'launched_year', 'launched_month', 'Diff

In [ ]:
combined_data['business_type'].value_counts()

Sole Proprietorship                    238089
Limited  Liability Company(LLC)        101472
Corporation                             70852
Independent Contractors                 62638
Self-Employed Individuals               60935
Subchapter S Corporation                27251
Non-Profit Organization                  6323
Single Member LLC                        5925
Partnership                              2339
Limited Liability Partnership            1613
Professional Association                 1234
501(c)3 – Non Profit                      346
Cooperative                               148
501(c)6 – Non Profit Membership           142
Non-Profit Childcare Center               101
Joint Venture                              92
Qualified Joint-Venture (spouses)          50
Trust                                      31
Employee Stock Ownership Plan(ESOP)        24
Tenant in Common                           14
Rollover as Business Start-Ups (ROB         2
Housing Co-op                     

In [ ]:
combined_data['business_type'].isna().sum()

41

In [ ]:
combined_data['business_type']=combined_data['business_type'].fillna("None")

In [ ]:
combined_data['business_type'].replace('Qualified Joint-Venture (spouses)','Others',inplace=True)
combined_data['business_type'].replace('Trust','Others',inplace=True)
combined_data['business_type'].replace("Employee Stock Ownership Plan(ESOP)",'Others',inplace=True)
combined_data['business_type'].replace("Tenant in Commo",'Others',inplace=True)
combined_data['business_type'].replace("Rollover as Business Start-Ups (ROB",'Others',inplace=True)
combined_data['business_type'].replace("Housing Co-op",'Others',inplace=True)
combined_data['business_type'].replace("501(c)19 – Non Profit Veterans",'Others',inplace=True)
combined_data['business_type'].replace("Tenant in Common",'Others',inplace=True)
  

In [ ]:
combined_data['business_type'].value_counts()

Sole Proprietorship                238089
Limited  Liability Company(LLC)    101472
Corporation                         70852
Independent Contractors             62638
Self-Employed Individuals           60935
Subchapter S Corporation            27251
Non-Profit Organization              6323
Single Member LLC                    5925
Partnership                          2339
Limited Liability Partnership        1613
Professional Association             1234
501(c)3 – Non Profit                  346
Cooperative                           148
501(c)6 – Non Profit Membership       142
Others                                123
Non-Profit Childcare Center           101
Joint Venture                          92
None                                   41
Name: business_type, dtype: int64

In [ ]:
combined_data['city'].nunique()

2785

In [ ]:
city_unique=combined_data['city'].unique()

In [ ]:
Top_5_city=['Atlanta', 'Columbus', 'Augusta', 'Macon', 'Savannah']
#, 'Athens', 'Sandy Springs', 'South Fulton', 'Roswell', 'Johns Creek']
a = (map(lambda x: x.lower(), Top_5_city))
Top_5_city_lower = list(a)
b = (map(lambda x: x.upper(), Top_5_city))
Top_5_city_upper = list(b)

In [ ]:
city_name=[]
for i in combined_data['city']:
  if (i in Top_5_city):
    city_name.append(i.upper())
  elif (i in Top_5_city_upper):
    city_name.append(i)
  elif (i in Top_5_city_lower):
    city_name.append(i.upper())
  else:
    city_name.append('Other')

In [ ]:
combined_data['city_name_top']=city_name


In [ ]:
combined_data['city_name']=city_name

In [ ]:
combined_data['city_name_top'].replace(Top_5_city_upper,'Top_city',inplace=True)


In [ ]:
combined_data['city_name_top'].value_counts()

Other       450404
Top_city    129260
Name: city_name_top, dtype: int64

In [ ]:
combined_data['servicing_lender_name'].nunique()

1186

In [ ]:
Top_20_servicing_lender=['Harvest Small Business Finance, LLC', 'Prestamos CDFI, LLC', 'Capital Plus Financial, LLC', 'Cross River Bank', 'Benworth Capital', 'Fountainhead SBF LLC', 'Customers Bank', 'Bank of America, National Association', 'Itria Ventures LLC', 'Truist Bank d/b/a Branch Banking & Trust Co',
             'Wells Fargo Bank, National Association', 'Synovus Bank']


In [ ]:
servicing_lender_name=[]
for i in combined_data['servicing_lender_name']:
  if (i in Top_20_servicing_lender):
    servicing_lender_name.append('Top')
  else:
    servicing_lender_name.append('Other')

In [ ]:
combined_data['servicing_lender_top']=servicing_lender_name

In [ ]:

combined_data['servicing_lender_state'].value_counts()

GA    111569
CA     78670
FL     71091
AZ     57364
TX     54650
NJ     45244
NC     37918
NY     24752
PA     23531
SD     14708
UT     11584
AL      8163
OH      7293
MS      6851
NE      4520
TN      4207
AR      2543
DE      2512
MA      2440
VA      1855
SC      1699
ID      1388
OK      1076
MO       721
NM       543
WV       506
IL       505
KS       332
MN       311
LA       150
NV       140
WI       101
KY        96
IN        86
IA        80
CT        76
MI        72
MD        60
CO        55
RI        39
WA        31
DC        24
OR        20
NH        19
ND        18
HI        17
MT        14
ME         8
VT         4
WY         3
GU         2
AS         2
AK         1
Name: servicing_lender_state, dtype: int64

In [ ]:
Top_10_servicing_lender_state=['GA', 'CA', 'FL', 'AZ', 'TX', 'NJ', 'NC', 'NY', 'PA', 'SD']
             #'UT', 'Synovus Bank', 'Kabbage, Inc.', 'United Community Bank', 'Ameris Bank', 'Square Capital, LLC', 'Regions Bank', 'JPMorgan Chase Bank, National Association', 'Readycap Lending, LLC', 'Celtic Bank Corporation']

In [ ]:
servicing_lender_state=[]
for i in combined_data['servicing_lender_state']:
  if (i in Top_10_servicing_lender_state):
    servicing_lender_state.append('Top')
  else:
    servicing_lender_state.append('Other')

In [ ]:
combined_data['servicing_lender_state_top']=servicing_lender_state

In [ ]:
combined_data['servicing_lender_state_top'].value_counts()

Top      519497
Other     60167
Name: servicing_lender_state_top, dtype: int64

In [ ]:
combined_data['servicing_lender_city'].value_counts()

Laguna Hills         62060
Phoenix              57324
Bedford              52255
TEANECK              36020
CHARLOTTE            35381
                     ...  
MERTZON                  1
HARRISBURG               1
BELLE CHASSE             1
BLACK RIVER FALLS        1
Columbus                 1
Name: servicing_lender_city, Length: 859, dtype: int64

In [ ]:
combined_data['servicing_lender_city']=combined_data['servicing_lender_city'].apply(lambda x: x.upper())

In [ ]:
combined_data['servicing_lender_city'].value_counts()

LAGUNA HILLS    62060
PHOENIX         57359
BEDFORD         52272
TEANECK         36020
CHARLOTTE       35381
                ...  
NEW MADISON         1
LAMONT              1
ELDORADO            1
SHANNON             1
CARIBOU             1
Name: servicing_lender_city, Length: 819, dtype: int64

In [ ]:
Top_2_servicing_lender_city=['Atlanta', 'Columbus', 'Los Angeles', 'San Diego', 'Jacksonville', 'Miami',
                             'Phoenix', 'Tucson', 'Houston', 'San Antonio', 'Newark', 'Jersey City', 'Charlotte', 'Raleigh'
                             , 'New York City', 'Buffalo', 'Philadelphia', 'Pittsburgh', 'Sioux Falls', 'Rapid City']
Top_2_servicing_lender_city = map(lambda x: x.upper(), Top_2_servicing_lender_city)
Top_2_servicing_lender_city=list(Top_2_servicing_lender_city)

In [ ]:
servicing_lender_city=[]
for i in combined_data['servicing_lender_city']:
  if (i in Top_2_servicing_lender_city):
    servicing_lender_city.append('Top')
  else:
    servicing_lender_city.append('Other')

In [ ]:
combined_data['servicing_lender_city_top']=servicing_lender_state

In [ ]:
combined_data['servicing_lender_city_top'].value_counts()

Top      519497
Other     60167
Name: servicing_lender_city_top, dtype: int64

In [ ]:

combined_data['business_age_description'].value_counts()

Existing or more than 2 years old         537065
New Business or 2 years or less            26609
Unanswered                                 15869
Startup, Loan Funds will Open Business        75
Change of Ownership                           46
Name: business_age_description, dtype: int64

In [ ]:
combined_data['project_city'].value_counts()


Atlanta                 71664
ATLANTA                 17635
Decatur                 13861
Lawrenceville           13249
Marietta                12769
                        ...  
1850 The Exchange SE        1
80 Townsend Circle          1
Covinton                    1
kENNESAW                    1
HOGANVILLE                  1
Name: project_city, Length: 2800, dtype: int64

In [ ]:
Top_5_project_city=['Atlanta', 'Sandy Springs', 'Roswell','Grayson', 'Berkeley Lake', 'Rest Haven','Marietta', 'Smyrna', 'Kennesaw']
#, 'Athens', 'Sandy Springs', 'South Fulton', 'Roswell', 'Johns Creek']
a = (map(lambda x: x.lower(), Top_5_project_city))
Top_5_project_city_lower = list(a)
b = (map(lambda x: x.upper(), Top_5_project_city))
Top_5_project_city_upper = list(b)

In [ ]:
project_city_name=[]
for i in combined_data['project_city']:
  if (i in Top_5_project_city):
    project_city_name.append(i.upper())
  elif (i in Top_5_project_city_upper):
    project_city_name.append(i)
  elif (i in Top_5_project_city_lower):
    project_city_name.append(i.upper())
  else:
    project_city_name.append('Other')

In [ ]:
combined_data['project_city_top']=project_city_name

In [ ]:
combined_data['project_city_top'].replace(Top_5_project_city_upper,'Top_city',inplace=True)

In [ ]:
combined_data['project_city_top'].value_counts()

Other       453576
Top_city    126088
Name: project_city_top, dtype: int64

In [ ]:
combined_data['project_county_name'].value_counts()

FULTON             101254
DEKALB              63266
GWINNETT            59226
COBB                45008
CLAYTON             30318
                    ...  
ALEXANDRIA CITY         1
BEAUFORT                1
ESCAMBIA                1
HAMILTON                1
CHARLOTTE               1
Name: project_county_name, Length: 171, dtype: int64

In [ ]:
combined_data['project_county_name'] = combined_data['project_county_name'].str.upper()

In [ ]:
combined_data['project_county_name'].value_counts()

FULTON             101254
DEKALB              63266
GWINNETT            59226
COBB                45008
CLAYTON             30318
                    ...  
ALEXANDRIA CITY         1
BEAUFORT                1
ESCAMBIA                1
HAMILTON                1
CHARLOTTE               1
Name: project_county_name, Length: 171, dtype: int64

In [ ]:
Top_5_project_county=['Fulton', 'Gwinnett', 'Cobb']
Top_5_project_county = map(lambda x: x.upper(), Top_5_project_county)
Top_5_project_county=list(Top_5_project_county)

In [ ]:
project_county=[]
for i in combined_data['project_county_name']:
  if (i in Top_5_project_county):
    project_county.append('Top')
  else:
    project_county.append('Other')

In [ ]:
combined_data['project_county_top']=project_county

In [ ]:
combined_data['project_county_top'].value_counts()

Other    374176
Top      205488
Name: project_county_top, dtype: int64

In [ ]:
combined_data['originating_lender_state'].value_counts()

GA    118833
CA     75410
FL     70950
AZ     57364
TX     54937
NJ     40100
NC     37918
NY     21012
PA     19943
UT     16032
SD     14709
AL      8169
OH      7293
MS      6966
MA      4530
TN      4209
VA      3507
NE      3422
AR      2543
DE      2512
SC      1804
ID      1388
OK      1076
MO       721
ME       635
NM       543
WV       506
IL       505
NV       425
MN       394
KS       332
LA       152
WI       101
KY        96
IN        86
IA        80
MI        76
CT        76
MD        60
CO        55
RI        39
WA        31
DC        24
OR        20
NH        19
ND        18
HI        17
MT        14
VT         4
WY         3
GU         2
AS         2
AK         1
Name: originating_lender_state, dtype: int64

In [ ]:
Top_10_lender_state=['GA', 'CA', 'FL', 'AZ', 'TX', 'NJ', 'NC', 'NY', 'PA', 'UT']

In [ ]:
lender_state=[]
for i in combined_data['originating_lender_state']:
  if (i in Top_10_lender_state):
    lender_state.append('Top')
  else:
    lender_state.append('Other')

In [ ]:
combined_data['lender_state_top']=lender_state

In [ ]:
combined_data['lender_state_top'].value_counts()

Top      512499
Other     67165
Name: lender_state_top, dtype: int64

In [ ]:
combined_data['originating_lender_city'].value_counts()

Phoenix         57324
Bedford         52249
Laguna Hills    40865
CHARLOTTE       35381
Miami           35095
                ...  
CORNING             1
WILLIAMSBURG        1
NEENAH              1
ROCKY MOUNT         1
Columbus            1
Name: originating_lender_city, Length: 863, dtype: int64

In [ ]:
combined_data['originating_lender_city']=combined_data['originating_lender_city'].apply(lambda x: x.upper())

In [ ]:
combined_data['originating_lender_city'].value_counts()

PHOENIX         57359
BEDFORD         52266
LAGUNA HILLS    40865
CHARLOTTE       35381
MIAMI           35163
                ...  
CORNING             1
WILLIAMSBURG        1
NEENAH              1
ROCKY MOUNT         1
CARIBOU             1
Name: originating_lender_city, Length: 823, dtype: int64

In [ ]:
Top_2_lender_city=['Atlanta', 'Columbus', 'Los Angeles', 'San Diego', 'Jacksonville', 'Miami',
                             'Phoenix', 'Tucson', 'Houston', 'San Antonio', 'Newark', 'Jersey City', 'Charlotte', 'Raleigh'
                             , 'New York City', 'Buffalo', 'Philadelphia', 'Pittsburgh', 'Salt Lake', 'West Valley']
Top_2_lender_city = map(lambda x: x.upper(), Top_2_servicing_lender_city)
Top_2_lender_city=list(Top_2_servicing_lender_city)



In [ ]:
lender_city=[]
for i in combined_data['originating_lender_city']:
  if (i in Top_2_lender_city):
    lender_city.append('Top')
  else:
    lender_city.append('Other')

In [ ]:
combined_data['lender_city_top']=servicing_lender_state

In [ ]:

combined_data['lender_servicing_lender_city'] = np.where(combined_data['originating_lender_city'] == combined_data['servicing_lender_city'], True, False)
  

In [ ]:
combined_data['lender_servicing_lender_city'].value_counts()

True     537703
False     41961
Name: lender_servicing_lender_city, dtype: int64

In [ ]:
combined_data['lender_servicing_lender_state'] = np.where(combined_data['originating_lender_state'] == combined_data['servicing_lender_state'], True, False)

In [ ]:
combined_data['lender_servicing_lender_state'].value_counts()

True     558909
False     20755
Name: lender_servicing_lender_state, dtype: int64

In [ ]:
combined_data['lender_servicing_lender_state_city']= np.where((combined_data['lender_servicing_lender_state'] == True)&(combined_data['lender_servicing_lender_city']==True), True, False)

In [ ]:
combined_data['lender_servicing_lender_state_city'].value_counts()

True     537703
False     41961
Name: lender_servicing_lender_state_city, dtype: int64

In [ ]:
tab=pd.crosstab(index=combined_data['Removed'], columns=combined_data['lender_servicing_lender_state_city'], margins=True)
tab

lender_servicing_lender_state_city,False,True,All
Removed,,,
False,41520,512308,553828
True,441,25395,25836
All,41961,537703,579664


In [ ]:
tab.columns = ['Top_launched_year_False','Top_launched_year_True',"rowtotal"]
tab.index= ['Removed_False','Removed_True',"coltotal"]

tab/tab.loc["coltotal"]

,Top_launched_year_False,Top_launched_year_True,rowtotal
Removed_False,0.98949,0.952771,0.955429
Removed_True,0.01051,0.047229,0.044571
coltotal,1.00000,1.000000,1.000000


In [ ]:
combined_data.columns

Index(['name', 'amount', 'state', 'address', 'city', 'zip', 'naics_code',
       'business_type', 'jobs_retained', 'date_approved', 'lender',
       'congressional_district', 'loan_number', 'sba_office_code',
       'processing_method', 'loan_status', 'term', 'sba_guaranty_percentage',
       'initial_approval_amount', 'current_approval_amount',
       'undisbursed_amount', 'servicing_lender_location_id',
       'servicing_lender_name', 'servicing_lender_address',
       'servicing_lender_city', 'servicing_lender_state',
       'servicing_lender_zip', 'rural_urban_indicator', 'hubzone_indicator',
       'business_age_description', 'project_city', 'project_county_name',
       'project_state', 'project_zip', 'originating_lender_city',
       'originating_lender_state', 'loan_status_date',
       'originating_lender_location_id', 'lmi_indicator', 'forgiveness_amount',
       'forgiveness_date', 'Removed', 'amount_per_join', 'Date_launched',
       'launched_year', 'launched_month', 'Diff

In [ ]:
categorical_data = pd.DataFrame(combined_data,columns=['launched_month','launched_year','naics_code_name', 'business_type', 'city_name_top', 'servicing_lender_top', 'servicing_lender_state_top', 'servicing_lender_city_top', 'project_city_top', 'project_county_top'
, 'lender_state_top', 'lender_city_top', 'lender_servicing_lender_state_city'])

In [ ]:
categorical_data

,launched_month,launched_year,naics_code_name,business_type,city_name_top,servicing_lender_top,servicing_lender_state_top,servicing_lender_city_top,project_city_top,project_county_top,lender_state_top,lender_city_top,lender_servicing_lender_state_city
0,2,2021,Manufacturing,Corporation,Top_city,Other,Other,Other,Other,Other,Other,Other,True
1,1,2021,"Professional, Scientific, and Technical Services",Corporation,Other,Top,Top,Top,Other,Top,Top,Top,True
2,3,2021,Construction,Corporation,Top_city,Top,Top,Top,Other,Other,Top,Top,True
3,2,2021,Manufacturing,Corporation,Other,Top,Top,Top,Other,Other,Top,Top,True
4,4,2020,Retail Trade,Partnership,Other,Other,Top,Top,Other,Other,Top,Top,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
579659,4,2021,Health Care and Social Assistance,Sole Proprietorship,Other,Top,Top,Top,Other,Other,Top,Top,True
579660,5,2021,Transportation and Warehousing,Independent Contractors,Top_city,Top,Top,Top,Top_city,Top,Top,Top,True
579661,4,2021,Transportation and Warehousing,Independent Contractors,Top_city,Top,Top,Top,Top_city,Top,Top,Top,True
579662,5,2021,Administrative and Support and Waste Managemen...,Limited Liability Company(LLC),Other,Top,Top,Top,Other,Other,Top,Top,True


In [ ]:
categorical_data_dummy = pd.get_dummies(categorical_data,
                     columns = ['launched_month','launched_year','naics_code_name', 'business_type', 'city_name_top', 'servicing_lender_top', 'servicing_lender_state_top', 'servicing_lender_city_top', 'project_city_top', 'project_county_top'
, 'lender_state_top', 'lender_city_top', 'lender_servicing_lender_state_city'])

In [ ]:
categorical_data_dummy

,launched_month_1,launched_month_2,launched_month_3,launched_month_4,launched_month_5,launched_month_6,launched_month_7,launched_month_8,launched_year_2020,launched_year_2021,...,project_city_top_Other,project_city_top_Top_city,project_county_top_Other,project_county_top_Top,lender_state_top_Other,lender_state_top_Top,lender_city_top_Other,lender_city_top_Top,lender_servicing_lender_state_city_False,lender_servicing_lender_state_city_True
0,0,1,0,0,0,0,0,0,0,1,...,1,0,1,0,1,0,1,0,0,1
1,1,0,0,0,0,0,0,0,0,1,...,1,0,0,1,0,1,0,1,0,1
2,0,0,1,0,0,0,0,0,0,1,...,1,0,1,0,0,1,0,1,0,1
3,0,1,0,0,0,0,0,0,0,1,...,1,0,1,0,0,1,0,1,0,1
4,0,0,0,1,0,0,0,0,1,0,...,1,0,1,0,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579659,0,0,0,1,0,0,0,0,0,1,...,1,0,1,0,0,1,0,1,0,1
579660,0,0,0,0,1,0,0,0,0,1,...,0,1,0,1,0,1,0,1,0,1
579661,0,0,0,1,0,0,0,0,0,1,...,0,1,0,1,0,1,0,1,0,1
579662,0,0,0,0,1,0,0,0,0,1,...,1,0,1,0,0,1,0,1,0,1


In [ ]:
df_all_cols = pd.concat([numeric_data, categorical_data_dummy], axis = 1)

In [ ]:
df_all_cols

,amount,amount_per_join,term,undisbursed_amount,Difference_current_initial_amount,launched_month_1,launched_month_2,launched_month_3,launched_month_4,launched_month_5,...,project_city_top_Other,project_city_top_Top_city,project_county_top_Other,project_county_top_Top,lender_state_top_Other,lender_state_top_Top,lender_city_top_Other,lender_city_top_Top,lender_servicing_lender_state_city_False,lender_servicing_lender_state_city_True
0,2000000.0,6802.72,60.0,0.0,0.0,0,1,0,0,0,...,1,0,1,0,1,0,1,0,0,1
1,1294555.0,20548.49,60.0,0.0,0.0,1,0,0,0,0,...,1,0,0,1,0,1,0,1,0,1
2,729509.0,6947.70,60.0,0.0,0.0,0,0,1,0,0,...,1,0,1,0,0,1,0,1,0,1
3,571193.4,11657.01,60.0,0.0,-0.6,0,1,0,0,0,...,1,0,1,0,0,1,0,1,0,1
4,472700.0,12775.68,24.0,0.0,0.0,0,0,0,1,0,...,1,0,1,0,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579659,1950.0,1950.00,60.0,1950.0,0.0,0,0,0,1,0,...,1,0,1,0,0,1,0,1,0,1
579660,1849.0,1849.00,60.0,1849.0,0.0,0,0,0,0,1,...,0,1,0,1,0,1,0,1,0,1
579661,1245.0,1245.00,60.0,1245.0,0.0,0,0,0,1,0,...,0,1,0,1,0,1,0,1,0,1
579662,1075.0,1075.00,60.0,1075.0,0.0,0,0,0,0,1,...,1,0,1,0,0,1,0,1,0,1


In [ ]:
target = pd.DataFrame(combined_data,
                     columns = ['Removed'])

In [ ]:
target_number = pd.get_dummies(target,
                     columns = ['Removed'])

In [ ]:
target_number

,Removed_False,Removed_True
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
579659,0,1
579660,0,1
579661,0,1
579662,0,1


In [ ]:
target_number.drop(['Removed_False'], axis=1, inplace=True)

In [ ]:
target_number

,Removed_True
0,0
1,0
2,0
3,0
4,0
...,...
579659,1
579660,1
579661,1
579662,1


In [ ]:
target_number.isna().sum()

Removed_True    0
dtype: int64

# Modeling

In [ ]:

logisticRegr = LogisticRegression()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_all_cols, target_number, test_size=0.3, random_state=42)

## Logistic regression

In [ ]:
logisticRegr = LogisticRegression()

In [ ]:
logisticRegr.fit(X_train,  y_train.values.ravel())

LogisticRegression()

In [ ]:
predictions = logisticRegr.predict(X_test)

In [ ]:
predictions

array([0, 0, 0, ..., 0, 1, 0], dtype=uint8)

In [ ]:
# Use score method to get accuracy of model
score = logisticRegr.score(X_test, y_test)
print(score)

0.9916676250718804


## Random Forest

In [ ]:
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf.fit(X_train,  y_train.values.ravel())

RandomForestRegressor(n_estimators=1000, random_state=42)

In [ ]:
predictions = rf.predict(X_test)


In [ ]:
y_pred=np.where(predictions > 0.5, 1, 0)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[165913    191]
 [   735   7061]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    166104
           1       0.97      0.91      0.94      7796

    accuracy                           0.99    173900
   macro avg       0.98      0.95      0.97    173900
weighted avg       0.99      0.99      0.99    173900

0.9946751006325475


## KNN

In [ ]:
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train,  y_train.values.ravel())
KNN_pred = classifier.predict(X_test)
KNN_pred

array([0, 0, 0, ..., 0, 1, 0], dtype=uint8)

In [ ]:

print(confusion_matrix(y_test,KNN_pred))
print(classification_report(y_test,KNN_pred))
print(accuracy_score(y_test, KNN_pred))

[[165994    110]
 [   728   7068]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    166104
           1       0.98      0.91      0.94      7796

    accuracy                           1.00    173900
   macro avg       0.99      0.95      0.97    173900
weighted avg       1.00      1.00      1.00    173900

0.9951811385853939


## XGBoost

In [ ]:
model = XGBClassifier()
model.fit(X_train,  y_train.values.ravel())
XGB_pred = model.predict(X_test)

In [ ]:

print(confusion_matrix(y_test,XGB_pred))
print(classification_report(y_test,XGB_pred))
print(accuracy_score(y_test, XGB_pred))

[[166001    103]
 [   737   7059]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    166104
           1       0.99      0.91      0.94      7796

    accuracy                           1.00    173900
   macro avg       0.99      0.95      0.97    173900
weighted avg       1.00      1.00      1.00    173900

0.9951696377228292
